Create Old Postring List

In [3]:
import re
import collections
import os
import time
import math
import random

In [4]:
indToDoc = {}
postingList = collections.defaultdict(list)
curDocID = 0
path = "practice3_collection"

In [5]:
def tokenize(f, path):
    #Returns [(term, pos), (term, pos) ...]
    terms = []
    with open(path + "/" + f) as file:
        line = file.readline()
        i = 0
        while line:
            # Regex to match only strings and spaces 
            line = re.sub(r'[^A-Za-z\s]+', '', line)
            for word in line.split():
                terms.append((word.lower(), i))
                i += 1
            line = file.readline() 
    return terms

In [6]:
def createPostingList(path):
    startTime = time.time()
    for f in os.listdir(path):
        # TODO: remove in real code
        global curDocID
        
        indToDoc[curDocID] = f

        # Returns [(word, pos), (word, pos) ...]
        terms = tokenize(f, path)

        # create map {word:[pos1, pos2, pos3]}
        wordToPos = collections.defaultdict(list)
        for word, pos in terms:
            wordToPos[word].append(pos)

        # append to posting list {term : [(docID1, [pos1, pos2, pos3, pos4])]}
        for term, arr in wordToPos.items():
            postingList[term].append((curDocID, wordToPos[term]))

        # For every file update id 
        curDocID += 1
    
    endTime = time.time()
    print(f"Index built in {endTime - startTime} seconds.")
    return postingList

In [7]:
result = createPostingList(path)

Index built in 0.0010848045349121094 seconds.


Create New Postring List

In [8]:
newPostingList = collections.defaultdict(list)
idToTerm = {}
curTermIndex = 0

In [9]:
stopList = set()
stopTuples = tokenize("stop-list.txt", ".")
for t, p in stopTuples:
    stopList.add(t)
print(stopList)

{'it', 'that', 'its', 'will', 'an', 'and', 'from', 'with', 'were', 'the', 'for', 'has', 'he', 'by', 'a', 'as', 'is', 'to', 'be', 'of', 'was', 'at', 'on', 'are', 'in'}


In [10]:
for k,v in result.items():
    # if k in stopList:
    #     continue
    idToTerm[curTermIndex] = k
    newPostingList[curTermIndex].append(math.log10(len(indToDoc)/len(v)))
    
    for docId, posList in v:
        newPostingList[curTermIndex].append((docId, 1 + math.log10(len(posList)), posList))
    curTermIndex += 1

In [11]:
print(newPostingList)

defaultdict(<class 'list'>, {0: [0.17609125905568124, (0, 1.3010299956639813, [0, 4]), (1, 1.3010299956639813, [0, 4])], 1: [0.17609125905568124, (0, 1.3010299956639813, [1, 6]), (1, 1.0, [1])], 2: [0.47712125471966244, (0, 1.0, [2])], 3: [0.17609125905568124, (0, 1.0, [3]), (1, 1.0, [3])], 4: [0.47712125471966244, (0, 1.0, [5])], 5: [0.17609125905568124, (1, 1.0, [2]), (2, 1.0, [2])], 6: [0.17609125905568124, (1, 1.0, [5]), (2, 1.0, [1])], 7: [0.47712125471966244, (2, 1.0, [0])]})


Convert Each Document To A Vector

In [18]:
docToVec = {}
for k in indToDoc.keys():
    docToVec[k] = len(idToTerm)*[float(0)]

for k, v in newPostingList.items():
    idf = v[0]
    for i in range(1, len(v)):
        t = v[i]
        doc, w = t[0], t[1]
        docToVec[doc][k] = idf*w

In [21]:
def cosine_similarity(v1,v2):
    "compute cosine similarity of v1 to v2: (v1 dot v2)/{||v1||*||v2||)"
    sumxx, sumxy, sumyy = 0, 0, 0
    for i in range(len(v1)):
        x = v1[i]; y = v2[i]
        sumxx += x*x
        sumyy += y*y
        sumxy += x*y
    if sumxx == 0 or sumyy == 0:
        return 0
    return sumxy/math.sqrt(sumxx*sumyy)

Assign leaders

In [ ]:
leadersArr = random.sample(range(0, len(indToDoc)), int(math.sqrt(len(indToDoc))))
leadersSet = set(leadersArr)
docToNearestLeader = {}
leaderAndFollowers = collections.defaultdict(list)

For each document find nearest leader

In [22]:
for doc in range(len(indToDoc)):
    docVec = docToVec[doc]
    distNearest = float("-inf")
    nearest = None
    for l in leadersArr:
        newDist = cosine_similarity(docVec, docToVec[l])
        if newDist >= distNearest:
            distNearest = newDist
            nearest = l
    docToNearestLeader[doc] = nearest

for k, v in docToNearestLeader.items():
    leaderAndFollowers[v].append(k)

Convert Query Into Vector

In [33]:
query_terms = ["a", "cat", "jumped"]

termToId = {}
for id, term in idToTerm.items():
    termToId[term] = id
    
query_terms = [word.lower() for word in query_terms]
termToFreq = {}
for t in query_terms:
    termToFreq[t] = termToFreq.get(t, 0) + 1

queryVector = len(idToTerm)*[float(0)]
for term, freq in termToFreq.items():
    if term in termToId:
        w = 1 + math.log10(freq)
        idf = newPostingList[termToId[term]][0]
        queryVector[termToId[term]] = w*idf

Find distance of query to leaders

In [27]:
leadersAndDist = []
for l in leadersArr:
    v = docToVec[l]
    distToQuery = cosine_similarity(v, queryVector)
    leadersAndDist.append((l, distToQuery))
leadersAndDist.sort(key=lambda x:x[1])

In [28]:
print(leadersAndDist)

[(1, 0.2742612593365751)]


For the cluster around nearest leader, sort all docs in cluster by distance to query vector and append them to result. If you run out of documents in current cluster, move on to next closest leader and that cluster. 

In [31]:
res = []
for l, dist in leadersAndDist:
    cluster = leaderAndFollowers[l]
    # Sort each element in cluster by distance to query vector
    clusterDistToQuery = []
    for n in cluster:
        dist = cosine_similarity(docToVec[n], queryVector)
        clusterDistToQuery.append((n, dist))
    clusterDistToQuery.sort(key=lambda x:x[1])
    res.extend([x[0] for x in clusterDistToQuery])
    if len(res) >= k:
        break
result = res[:k]

In [34]:
print(result)

[0, 1]
